In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(1969, size1)
        self.conv2 = GCNConv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [ ]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_expression/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_expression/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_expression/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_expression/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1) * 1000
    Y_valid = pd.read_csv("../../result/input_perturb_expression/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1) * 1000
    Y_test = pd.read_csv("../../result/input_perturb_expression/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1) * 1000
    Y_test_gene = pd.read_csv("../../result/input_perturb_expression/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_phyloP/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_phyloP/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF", "NOTEARS_thresholdAdaptive"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_phyloP/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_expression/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_expression/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-14 15:06:46,808] A new study created in memory with name: no-name-e75cb79a-e0c0-4288-a200-efb22141318e
[I 2023-12-14 15:06:51,135] Trial 0 finished with value: 7.439224720001221 and parameters: {'size1': 594, 'size2': 478, 'size3': 194, 'num_epochs': 93, 'lr': 0.0038944293122681905}. Best is trial 0 with value: 7.439224720001221.
[I 2023-12-14 15:06:51,639] Trial 1 finished with value: 6.966238975524902 and parameters: {'size1': 460, 'size2': 79, 'size3': 207, 'num_epochs': 130, 'lr': 0.00836193902393883}. Best is trial 1 with value: 6.966238975524902.
[I 2023-12-14 15:06:51,834] Trial 2 finished with value: 6.86529541015625 and parameters: {'size1': 90, 'size2': 323, 'size3': 161, 'num_epochs': 52, 'lr': 0.005961091269959644}. Best is trial 2 with value: 6.86529541015625.
[I 2023-12-14 15:06:52,540] Trial 3 finished with value: 6.8607940673828125 and parameters: {'size1': 1009, 'size2': 10, 'size3': 70, 'num_epochs': 150, 'lr': 0.001853346379811491}. Best is trial 3 with va

[I 2023-12-14 15:07:06,195] Trial 2 finished with value: 6.803345680236816 and parameters: {'size1': 1008, 'size2': 67, 'size3': 58, 'num_epochs': 153, 'lr': 0.0037292245794793857}. Best is trial 1 with value: 5.981274604797363.
[I 2023-12-14 15:07:06,466] Trial 3 finished with value: 7.040299892425537 and parameters: {'size1': 606, 'size2': 321, 'size3': 119, 'num_epochs': 61, 'lr': 0.005554274068043869}. Best is trial 1 with value: 5.981274604797363.
[I 2023-12-14 15:07:06,857] Trial 4 finished with value: 6.708124160766602 and parameters: {'size1': 388, 'size2': 99, 'size3': 344, 'num_epochs': 101, 'lr': 0.009629955487190782}. Best is trial 1 with value: 5.981274604797363.
[I 2023-12-14 15:07:06,885] Trial 5 pruned. 
[I 2023-12-14 15:07:06,901] Trial 6 pruned. 
[I 2023-12-14 15:07:06,952] Trial 7 pruned. 
[I 2023-12-14 15:07:07,172] Trial 8 finished with value: 6.8130388259887695 and parameters: {'size1': 533, 'size2': 89, 'size3': 220, 'num_epochs': 48, 'lr': 0.0022268174312978263}

[I 2023-12-14 15:07:17,616] Trial 17 pruned. 
[I 2023-12-14 15:07:17,667] Trial 18 pruned. 
[I 2023-12-14 15:07:17,711] Trial 19 pruned. 
[I 2023-12-14 15:07:18,122] A new study created in memory with name: no-name-f7e61c05-4b03-4e69-9212-f93976c5573c
[I 2023-12-14 15:07:18,240] Trial 0 finished with value: 6.744766712188721 and parameters: {'size1': 290, 'size2': 239, 'size3': 260, 'num_epochs': 29, 'lr': 0.0023590324665093768}. Best is trial 0 with value: 6.744766712188721.
[I 2023-12-14 15:07:18,901] Trial 1 finished with value: 7.026704788208008 and parameters: {'size1': 250, 'size2': 454, 'size3': 116, 'num_epochs': 177, 'lr': 0.005635543696209818}. Best is trial 0 with value: 6.744766712188721.
[I 2023-12-14 15:07:19,443] Trial 2 finished with value: 7.047384262084961 and parameters: {'size1': 434, 'size2': 89, 'size3': 308, 'num_epochs': 140, 'lr': 0.001375726173312416}. Best is trial 0 with value: 6.744766712188721.
[I 2023-12-14 15:07:19,754] Trial 3 finished with value: 6.903

[I 2023-12-14 15:07:28,417] Trial 2 finished with value: 6.495726585388184 and parameters: {'size1': 748, 'size2': 475, 'size3': 414, 'num_epochs': 66, 'lr': 0.005601667768587455}. Best is trial 0 with value: 6.014908313751221.
[I 2023-12-14 15:07:28,818] Trial 3 finished with value: 6.683291912078857 and parameters: {'size1': 785, 'size2': 33, 'size3': 214, 'num_epochs': 84, 'lr': 0.0032612584059502955}. Best is trial 0 with value: 6.014908313751221.
[I 2023-12-14 15:07:28,934] Trial 4 finished with value: 5.984090328216553 and parameters: {'size1': 550, 'size2': 421, 'size3': 511, 'num_epochs': 23, 'lr': 0.003572437885718265}. Best is trial 4 with value: 5.984090328216553.
[I 2023-12-14 15:07:28,951] Trial 5 pruned. 
[I 2023-12-14 15:07:28,970] Trial 6 pruned. 
[I 2023-12-14 15:07:29,448] Trial 7 finished with value: 6.9109416007995605 and parameters: {'size1': 314, 'size2': 65, 'size3': 73, 'num_epochs': 122, 'lr': 0.004341108911737982}. Best is trial 4 with value: 5.984090328216553

[I 2023-12-14 15:07:57,069] Trial 2 finished with value: 7.060296535491943 and parameters: {'size1': 705, 'size2': 21, 'size3': 372, 'num_epochs': 145, 'lr': 0.0069174781666314095}. Best is trial 0 with value: 6.524258136749268.
[I 2023-12-14 15:07:57,458] Trial 3 finished with value: 6.013825416564941 and parameters: {'size1': 219, 'size2': 449, 'size3': 289, 'num_epochs': 41, 'lr': 0.0012225118243550355}. Best is trial 3 with value: 6.013825416564941.
[I 2023-12-14 15:07:58,145] Trial 4 finished with value: 6.355769634246826 and parameters: {'size1': 462, 'size2': 216, 'size3': 94, 'num_epochs': 70, 'lr': 0.005897855347206271}. Best is trial 3 with value: 6.013825416564941.
[I 2023-12-14 15:07:58,171] Trial 5 pruned. 
[I 2023-12-14 15:07:58,750] Trial 6 finished with value: 6.183385372161865 and parameters: {'size1': 52, 'size2': 304, 'size3': 179, 'num_epochs': 94, 'lr': 0.009033934878848652}. Best is trial 3 with value: 6.013825416564941.
[I 2023-12-14 15:08:00,571] Trial 7 finishe

...

[I 2023-12-14 19:23:27,913] Trial 10 pruned. 
[I 2023-12-14 19:23:27,957] Trial 11 pruned. 
[I 2023-12-14 19:23:29,322] Trial 12 finished with value: 6.222380638122559 and parameters: {'size1': 211, 'size2': 136, 'size3': 109, 'num_epochs': 67, 'lr': 0.003655459113457336}. Best is trial 4 with value: 6.0363569259643555.
[I 2023-12-14 19:23:29,377] Trial 13 pruned. 
[I 2023-12-14 19:23:29,434] Trial 14 pruned. 
[I 2023-12-14 19:23:29,488] Trial 15 pruned. 
[I 2023-12-14 19:23:29,533] Trial 16 pruned. 
[I 2023-12-14 19:23:29,597] Trial 17 pruned. 
[I 2023-12-14 19:23:34,310] Trial 18 finished with value: 6.3325886726379395 and parameters: {'size1': 461, 'size2': 438, 'size3': 60, 'num_epochs': 94, 'lr': 0.0059886318924217705}. Best is trial 4 with value: 6.0363569259643555.
[I 2023-12-14 19:23:34,356] Trial 19 pruned. 
[I 2023-12-14 19:23:35,928] A new study created in memory with name: no-name-aa1f5392-2078-4cc4-a815-6cb162e270e0
[I 2023-12-14 19:23:38,001] Trial 0 finished with value: 

[I 2023-12-14 19:25:17,558] Trial 12 finished with value: 6.0230584144592285 and parameters: {'size1': 49, 'size2': 106, 'size3': 173, 'num_epochs': 18, 'lr': 0.003913298052243043}. Best is trial 2 with value: 5.83207368850708.
[I 2023-12-14 19:25:17,728] Trial 13 finished with value: 5.860191822052002 and parameters: {'size1': 627, 'size2': 156, 'size3': 85, 'num_epochs': 3, 'lr': 0.002902332385387405}. Best is trial 2 with value: 5.83207368850708.
[I 2023-12-14 19:25:19,623] Trial 14 finished with value: 6.183849811553955 and parameters: {'size1': 673, 'size2': 197, 'size3': 21, 'num_epochs': 37, 'lr': 0.002514658565125678}. Best is trial 2 with value: 5.83207368850708.
[I 2023-12-14 19:25:19,706] Trial 15 pruned. 
[I 2023-12-14 19:25:19,794] Trial 16 finished with value: 5.836495399475098 and parameters: {'size1': 809, 'size2': 21, 'size3': 80, 'num_epochs': 1, 'lr': 0.002562819507282428}. Best is trial 2 with value: 5.83207368850708.
[I 2023-12-14 19:25:19,882] Trial 17 pruned. 
[I

[I 2023-12-14 19:26:11,024] Trial 5 finished with value: 6.051504135131836 and parameters: {'size1': 829, 'size2': 85, 'size3': 414, 'num_epochs': 38, 'lr': 0.0031607749002206453}. Best is trial 1 with value: 5.861398696899414.
[I 2023-12-14 19:26:11,064] Trial 6 pruned. 
[I 2023-12-14 19:26:11,110] Trial 7 pruned. 
[I 2023-12-14 19:26:11,177] Trial 8 pruned. 
[I 2023-12-14 19:26:11,255] Trial 9 pruned. 
[I 2023-12-14 19:26:11,294] Trial 10 pruned. 
[I 2023-12-14 19:26:11,381] Trial 11 pruned. 
[I 2023-12-14 19:26:11,443] Trial 12 pruned. 
[I 2023-12-14 19:26:11,519] Trial 13 pruned. 
[I 2023-12-14 19:26:11,587] Trial 14 pruned. 
[I 2023-12-14 19:26:11,668] Trial 15 pruned. 
[I 2023-12-14 19:26:11,732] Trial 16 pruned. 
[I 2023-12-14 19:26:11,802] Trial 17 pruned. 
[I 2023-12-14 19:26:11,883] Trial 18 pruned. 
[I 2023-12-14 19:26:11,958] Trial 19 pruned. 
[I 2023-12-14 19:26:12,843] A new study created in memory with name: no-name-b8e32f8e-bd73-4767-a1d1-c968746244ff
[I 2023-12-14 19:2

[I 2023-12-14 19:27:21,484] Trial 17 pruned. 
[I 2023-12-14 19:27:21,528] Trial 18 pruned. 
[I 2023-12-14 19:27:24,011] Trial 19 finished with value: 6.271428108215332 and parameters: {'size1': 397, 'size2': 310, 'size3': 351, 'num_epochs': 98, 'lr': 0.0037707249399431394}. Best is trial 11 with value: 5.818294048309326.
[I 2023-12-14 19:27:24,064] A new study created in memory with name: no-name-7392d91a-0671-4113-92d2-311326052a3c
[I 2023-12-14 19:27:24,321] Trial 0 finished with value: 5.86736536026001 and parameters: {'size1': 129, 'size2': 268, 'size3': 421, 'num_epochs': 17, 'lr': 0.008945896187101855}. Best is trial 0 with value: 5.86736536026001.
[I 2023-12-14 19:27:27,162] Trial 1 finished with value: 6.048405647277832 and parameters: {'size1': 32, 'size2': 386, 'size3': 465, 'num_epochs': 184, 'lr': 0.007559971632134916}. Best is trial 0 with value: 5.86736536026001.
[I 2023-12-14 19:27:29,355] Trial 2 finished with value: 6.214094161987305 and parameters: {'size1': 242, 'siz

[I 2023-12-14 19:28:41,630] Trial 17 pruned. 
[I 2023-12-14 19:28:41,699] Trial 18 pruned. 
[I 2023-12-14 19:28:41,755] Trial 19 pruned. 
[I 2023-12-14 19:28:41,854] A new study created in memory with name: no-name-5e205c30-227b-4f83-adb3-11ca520c1df8
